In [1]:
# Code for ETL operations on Country-GDP data

# Importing the required libraries
import sqlite3
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime
import requests 
import re 



def log_progress(message):
    ''' This function logs the mentioned message of a given stage of the
    code execution to a log file. Function returns nothing'''
    # Getting the current date and time
    dt = datetime.now()
    # getting the timestamp
    ts = datetime.timestamp(dt)

    with open('code_log.txt', 'a') as log_file:
        log_file.write(f'{ts} : {message}\n')

''' Here, you define the required entities and call the relevant
functions in the correct order to complete the project. Note that this
portion is not inside any function.'''

log_progress('Preliminaries complete. Initiating ETL process')





In [2]:
 

def extract(url, table_attribs=None):
    ''' This function aims to extract the required
    information from the website and save it to a data frame. The
    function returns the data frame for further processing. '''
    # request for HTML document of given url 
    response = requests.get(url) 
      
    # text to BS4
    soup = BeautifulSoup(response.text , 'html.parser') 
    table = soup.find_all('table')
    df = pd.read_html(str(table))[0]

    return df


df = extract('https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks')
log_progress('Data extraction complete. Initiating Transformation process')
df

,Rank,Bank name,Market cap(US$ billion)
0,1,JPMorgan Chase,432.92
1,2,Bank of America,231.52
2,3,Industrial and Commercial Bank of China,194.56
3,4,Agricultural Bank of China,160.68
4,5,HDFC Bank,157.91
5,6,Wells Fargo,155.87
6,7,HSBC Holdings PLC,148.90
7,8,Morgan Stanley,140.83
8,9,China Construction Bank,139.82
9,10,Bank of China,136.81


In [3]:
df.rename(columns={'Market cap(US$ billion)':'MC_USD_Billion'}, inplace=True)
df

,Rank,Bank name,MC_USD_Billion
0,1,JPMorgan Chase,432.92
1,2,Bank of America,231.52
2,3,Industrial and Commercial Bank of China,194.56
3,4,Agricultural Bank of China,160.68
4,5,HDFC Bank,157.91
5,6,Wells Fargo,155.87
6,7,HSBC Holdings PLC,148.90
7,8,Morgan Stanley,140.83
8,9,China Construction Bank,139.82
9,10,Bank of China,136.81


In [4]:
def transform(df, csv_path):
    ''' This function accesses the CSV file for exchange rate
    information, and adds three columns to the data frame, each
    containing the transformed version of Market Cap column to
    respective currencies'''
    # Read the CSV file
    exchange_rate_df = pd.read_csv(csv_path)
    # convert the datafram into a dictionary
    exchange_rate = exchange_rate_df.set_index('Currency').to_dict()['Rate']

    df.rename(columns={'Market cap(US$ billion)':'MC_USD_Billion'}, inplace=True)

    currency = ['EUR', 'GBP', 'INR']
    new_column = ['MC_GBP_Billion', 'MC_EUR_Billion', 'MC_INR_Billion']
    for curr, col in zip(currency, new_column):
        df[col] = [np.round(x*exchange_rate[curr],2) for x in df['MC_USD_Billion']]

    return df


transform(df, 'exchange_rate.csv')
log_progress('Data transformation complete. Initiating Loading process')



In [5]:
df['MC_EUR_Billion'][4]

126.33

In [6]:
def load_to_csv(df, output_path):
    ''' This function saves the final data frame as a CSV file in
    the provided path. Function returns nothing.'''
    df.to_csv(output_path)


load_to_csv(df, './Largest_banks_data.csv')
log_progress('Data saved to CSV file')

In [7]:
def load_to_db(df, sql_connection, table_name):
    ''' This function saves the final data frame to a database
    table with the provided name. Function returns nothing.'''
    df.to_sql(name=table_name, con=sql_connection)

#init sql connection
# Create a SQL connection to our SQLite database
conn = sqlite3.connect("Banks.db")

log_progress('SQL Connection initiated')
load_to_db(df, conn, "Largest_banks")
log_progress('Data loaded to Database as a table, Executing queries')

c:\Users\saman\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\generic.py:2882: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


In [8]:
def run_query(query_statement, sql_connection):
    ''' This function runs the query on the database table and
    prints the output on the terminal. Function returns nothing. '''
    res = sql_connection.execute(query_statement)
    print (res.fetchone())

cur = conn.cursor()
run_query("SELECT * FROM Largest_banks" , cur)
run_query("SELECT AVG(MC_GBP_Billion) FROM Largest_banks" , cur)
run_query('SELECT "Bank name" from Largest_banks LIMIT 5' , cur)

log_progress('Process Complete')
#close the sql connection
# conn.close()
log_progress('Server Connection closed')

(0, 1, 'JPMorgan Chase', 432.92, 402.62, 346.34, 35910.71)
(176.68300000000005,)
('JPMorgan Chase',)
